## Informações sobre o DataSet

Este dataset tem registros de uma matriz de 8 sensores de gaz MOX, além de temperatura e Humidade. Estes sensoresforam expostos a atividade de dois diferentes estímulos: banana e vinho. As respostas para os estimulos de vinho e banana foram registradas colocando-se os estimuladores próximos aos sensores. A duração de cada estímulo variou entre 7 minutos a duas horas, sendo a media de duração de 42 minutos. Este conjunto de dados contem um conjunto de séries temporais de tres diferentes condições: estímulos por vinho, banana e atividade de backgroud/segundo plano. Existem 36 induções por vinho, 33 de banana e 31 de atividades de background/segundo plano. Uma possível aplicação e a discriminação entre vinho, banana e atividade de background/segundo plano.

Este dataset é composto de dois arquivos HT_sensor_dataset.dat (zipado), onde as séries temporais são armazendadas, além do HT_Sensor_metadata.dat, onde metadados para cada indução são armazenados. Cada indução é unicamqente identificada pelo campo "id" em ambos os arquivos. Por isso, metadados para uma particular indução podem ser facilmente localizados, correspondondo-se as colunas id de cada arquivo.

#### Reference: 
http://archive.ics.uci.edu/ml/datasets/Gas+sensors+for+home+activity+monitoring<br>
https://moisesomena.gitbooks.io/dw-bi/content/sensors-data.html

In [1]:
import timeit

In [2]:
#!pip install pandas

In [3]:
import pandas as pd

In [4]:
#!pip install sqlalchemy

In [5]:
from sqlalchemy import create_engine

In [6]:
import pandas as pd
from pymongo import MongoClient

In [7]:
HT_Sensor_metadata = pd.read_table("HT_Sensor_metadata.dat")
HT_Sensor_metadata.head()

,id,date,Unnamed: 2,class,t0,dt
0,0,07-04-15,banana,13.49,1.64,NaN
1,1,07-05-15,wine,19.61,0.54,NaN
2,2,07-06-15,wine,19.99,0.66,NaN
3,3,07-09-15,banana,6.49,0.72,NaN
4,4,07-09-15,wine,20.07,0.53,NaN


In [8]:
#dados_sensores.columns = ['a', 'b', 'c', 'd', 'e','f']
#dados_sensores

In [9]:
HT_Sensor_metadata.rename(columns={'Unnamed: 2': 'class', 'class': 't0','t0': 'dt','dt': 'discarte'}, inplace=True)

In [10]:
colunas=HT_Sensor_metadata.columns[:-1]
lista_colunas = list(colunas)
lista_colunas

['id', 'date', 'class', 't0', 'dt']

In [11]:
hts_metadata_ajust=HT_Sensor_metadata[lista_colunas].set_index('id')

In [12]:
hts_metadata_ajust.head()

,date,class,t0,dt
id,,,,
0,07-04-15,banana,13.49,1.64
1,07-05-15,wine,19.61,0.54
2,07-06-15,wine,19.99,0.66
3,07-09-15,banana,6.49,0.72
4,07-09-15,wine,20.07,0.53


In [13]:
HT_Sensor_dataset=pd.read_table('HT_Sensor_dataset.dat',sep='\s+')

In [14]:
HT_Sensor_dataset.head()

,id,time,R1,R2,R3,R4,R5,R6,R7,R8,Temp.,Humidity
0,0,-0.999750,12.8621,10.3683,10.4383,11.6699,13.4931,13.3423,8.04169,8.73901,26.2257,59.0528
1,0,-0.999472,12.8617,10.3682,10.4375,11.6697,13.4927,13.3412,8.04133,8.73908,26.2308,59.0299
2,0,-0.999194,12.8607,10.3686,10.4370,11.6696,13.4924,13.3405,8.04101,8.73915,26.2365,59.0093
3,0,-0.998916,12.8602,10.3686,10.4370,11.6697,13.4921,13.3398,8.04086,8.73936,26.2416,58.9905
4,0,-0.998627,12.8595,10.3688,10.4374,11.6699,13.4919,13.3390,8.04087,8.73986,26.2462,58.9736


In [15]:
hts_data_ajust=HT_Sensor_dataset.set_index('id')

In [16]:
hts_data_ajust.head()

,time,R1,R2,R3,R4,R5,R6,R7,R8,Temp.,Humidity
id,,,,,,,,,,,
0,-0.999750,12.8621,10.3683,10.4383,11.6699,13.4931,13.3423,8.04169,8.73901,26.2257,59.0528
0,-0.999472,12.8617,10.3682,10.4375,11.6697,13.4927,13.3412,8.04133,8.73908,26.2308,59.0299
0,-0.999194,12.8607,10.3686,10.4370,11.6696,13.4924,13.3405,8.04101,8.73915,26.2365,59.0093
0,-0.998916,12.8602,10.3686,10.4370,11.6697,13.4921,13.3398,8.04086,8.73936,26.2416,58.9905
0,-0.998627,12.8595,10.3688,10.4374,11.6699,13.4919,13.3390,8.04087,8.73986,26.2462,58.9736


In [17]:
#!pip install pymongo

In [18]:
client = MongoClient()
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [19]:
client = MongoClient('mongodb://127.0.0.1:27017/')
client

MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True)

In [20]:
hts_metadata_ajust_dict=hts_metadata_ajust.to_dict('records')

In [21]:
hts_metadata_ajust_dict[0:1]

[{'class': 'banana', 'date': '07-04-15', 'dt': 1.64, 't0': 13.49}]

In [22]:
db = client.gas_sensors
db.metadata.insert_many(hts_metadata_ajust_dict)

In [23]:
hts_data_ajust.rename(columns={'Temp.': 'Temp'}, inplace=True)

In [24]:
hts_data_ajust_dict= hts_data_ajust.to_dict('records')

In [25]:
hts_data_ajust_dict[0:1]

[{'Humidity': 59.052799999999998,
  'R1': 12.8621,
  'R2': 10.3683,
  'R3': 10.4383,
  'R4': 11.6699,
  'R5': 13.4931,
  'R6': 13.3423,
  'R7': 8.0416899999999991,
  'R8': 8.7390100000000004,
  'Temp': 26.2257,
  'time': -0.99975000000000003}]

In [26]:
db.data.insert_many(hts_data_ajust_dict)

## Obtendo amostra com junção dos dados

In [27]:
sample_metadata=hts_metadata_ajust.sample(n=30)

In [28]:
sample_data = hts_data_ajust.sample(3000)

In [29]:
meta_join_data = sample_metadata.join(sample_data)

In [30]:
meta_join_data.rename(columns={'Temp.': 'Temp'}, inplace=True)

In [31]:
meta_join_data_dict = meta_join_data.to_dict('records')

In [32]:
meta_join_data_dict[0:2]

[{'Humidity': 57.0357,
  'R1': 13.1391,
  'R2': 10.166,
  'R3': 10.2088,
  'R4': 11.447,
  'R5': 12.3649,
  'R6': 14.4145,
  'R7': 7.55772,
  'R8': 8.15746,
  'Temp': 25.6002,
  'class': 'wine',
  'date': '07-05-15',
  'dt': 0.54,
  't0': 19.61,
  'time': -0.042957999999999996},
 {'Humidity': 57.0958,
  'R1': 13.2655,
  'R2': 10.5688,
  'R3': 10.5666,
  'R4': 11.8515,
  'R5': 13.2024,
  'R6': 15.9863,
  'R7': 5.711790000000001,
  'R8': 6.46101,
  'Temp': 25.6339,
  'class': 'wine',
  'date': '07-05-15',
  'dt': 0.54,
  't0': 19.61,
  'time': 1.085949}]

In [33]:
db.join_sensors.insert_many(meta_join_data_dict)


In [34]:
meta_join_data.sample(n=30)

,date,class,t0,dt,time,R1,R2,R3,R4,R5,R6,R7,R8,Temp,Humidity
id,,,,,,,,,,,,,,,
86,08-27-15,background,12.53,0.50,-0.780313,11.9168,8.64768,8.79013,9.79271,6.92495,15.4909,5.77476,6.95213,28.1230,64.4120
57,08-30-15,banana,12.61,1.68,1.642699,10.1295,7.66304,7.42418,8.77569,9.30571,18.9132,3.11502,3.49914,24.4917,53.5147
3,07-09-15,banana,6.49,0.72,-0.663811,10.7933,7.15982,7.27759,8.16353,10.37700,11.2857,4.70089,4.60761,26.3391,73.8198
86,08-27-15,background,12.53,0.50,1.371038,12.0717,9.12517,9.27674,10.31450,13.85600,15.6950,5.73410,6.77373,28.3252,62.4132
3,07-09-15,banana,6.49,0.72,-0.660164,10.7789,7.13178,7.25718,8.12827,10.37030,11.2675,4.68222,4.59212,26.3181,73.4412
1,07-05-15,wine,19.61,0.54,-0.207320,13.2467,9.99062,9.99838,11.21910,12.44260,14.5079,7.49712,8.09307,25.5346,57.1150
73,08-07-15,background,13.16,0.61,0.658722,12.3321,10.16630,10.33920,11.53720,10.33090,15.3701,5.49277,5.63883,26.6926,50.3290
84,08-25-15,background,13.27,0.62,-0.298869,12.4548,10.04120,10.17550,11.31690,10.56120,16.5290,7.06754,7.35210,27.1392,65.0069
86,08-27-15,background,12.53,0.50,-0.475378,11.9786,8.80814,8.94833,9.95993,9.55575,15.5478,5.72461,6.88063,28.2466,63.9837


References:<br>
    http://ec2-54-218-106-48.us-west-2.compute.amazonaws.com/moschetti.org/rants/mongopandas.html<br>
    http://api.mongodb.com/python/current/tutorial.html<br>
    http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_dict.html?highlight=to_dict#pandas.DataFrame.to_dict<br>
    https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_dict.html<br>
    https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_json.html<br>
    https://stackoverflow.com/questions/20167194/insert-a-pandas-dataframe-into-mongodb-using-pymongo<br>
    http://api.mongodb.com/python/current/api/pymongo/collection.html#pymongo.collection.Collection.insert<br>
    